In [1]:
import numpy as np
import pandas as pd


Defining the Sigmoid function

\begin{align}
\sigma{(x)} & = 1/1+e^{-x}
\end{align}

In [2]:
def sigmoid(x):
    return 1/1+np.exp(-x)

Taking the derivative of sigmoid function we get
\begin{equation*}
\frac{\partial }{\partial x}  \sigma{(x)}  = {\sigma{(x)}} * (1-\sigma{(x)})
\end{equation*}

In [3]:
def sigmoid_prime(x):
    return sigmoid(x) * 1-(sigmoid(x))

In [4]:
# Input data
x = np.array([0.1, 0.3])
# Target
y = 0.2
# Input to output weights
weights = np.array([-0.8, 0.5])

In [5]:
learnrate = 0.5

In [6]:
# the linear combination performed by the node (h in f(h) and f'(h))
h = x[0]*weights[0] + x[1]*weights[1]

The output of neural network would be

In [7]:
nn_output = sigmoid(h)

 \begin{align}
 Error =  y - {y\widehat{}}
         \end{align}

Here,
\begin{equation*}
y\widehat{}
\end{equation*}
is actually nn_output

In [8]:
error = y - nn_output

Gradient of output can be found by passing the output(*h*) through the derivative of sigmoid

In [9]:
output_grad = sigmoid_prime(h)

 \begin{align}
 \delta =  Error * output grad
         \end{align}

In [10]:
error_term = error*output_grad

Now Let us perfrom Gradient Decent Step

In [11]:
del_w = [ learnrate * error_term * x[0],
          learnrate * error_term * x[1]]

print('Neural Network output:')
print(nn_output)
print('Amount of Error:')
print(error)
print('Change in Weights:')
print(del_w)

Now let us try to implement the same for 4 values

In [12]:
learnrate = 0.5
x = np.array([1, 2, 3, 4])
y = np.array(0.5)
# Initial weights
w = np.array([0.5, -0.5, 0.3, 0.1])

In [13]:
h = np.dot(x,w)

In [14]:
nn_output=sigmoid(h)

In [15]:
error = y - nn_output

In [16]:
output_grad = sigmoid_prime(h)


In [17]:
error_term = error * output_grad


In [18]:
del_w = learnrate * error_term * x


In [19]:
print('Neural Network output:')
print(nn_output)
print('Amount of Error:')
print(error)
print('Change in Weights:')
print(del_w)

Neural Network output:
1.4493289641172216
Amount of Error:
-0.9493289641172216
Change in Weights:
[-0. -0. -0. -0.]


## Implementing Gradient Descent in Actual Dataset

Now we have seen how to update the weights for a single iteration but we need to do the same for multiple epochs
Let us use this [http://www.ats.ucla.edu/stat/data/binary.cs](https://stats.idre.ucla.edu/stat/data/binary.csv) dataset for example.

The dataset given here have three input features GRE score GPA and rank of the undergrad school.
The goal here is to predict if the student gets accepted to the university with the given criterion we will use sigmoid funtion here also for output activation.

## Data Cleanup

If you look at the dataset now you will notice that the column rank is not hot encoded correctly,
Rank 2 is not twice Rank 1 or Rank 3 is not thrice Rank 1 or anything like that.

For this we need to use dummyvariables and split the ranks into four columns and then use 0 or 1 to represent them.

First lets read the data using pandas

In [20]:
admissions = pd.read_csv('binary.csv')


Now we can do the encoding in rank

In [21]:
# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

We now have to standadize the gre and gpa since they are fairly large values. 
We are using sigmoid activation function and this would squash fairly large and small values.
The gradient of this would become 0 and the gradient descent step would go to zero and die off.
We should standardize the values by keeping a mean of 0 and standard deviation of 1.


In [22]:
weights = np.array([0,0])

In [23]:
# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

/Users/abhijitramesh/miniconda3/envs/gradientdescent/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


## Mean Square Error

Since we are using large data here using, SSE would be hard since the gradient would decrease very fast and the training won't be proper. We have to compensate this by using a very small learning rate. We can also use Mean Square Error menthod and the learning rate would be in the range of 0.01 and 0.001.

 \begin{align}
 Error = \frac{1}{2m} & \sum_{\mu} ({y^{\mu} - y\widehat{}^\mu})^2
         \end{align}

What we are going to repeat the following steps for e epohs:

1. Set the change in weight to zero \begin{equation*} \nabla w_i = 0 \end{equation*}
2. For record in training data:

    * We calculate the output for \begin{equation*} y\widehat{} = f(\sum_i w_ix_i) \end{equation*}
    * Calculate the error term for output \begin{equation*} \delta = (y - y\widehat{}) * f^| (\sum_i w_i x_i)\end{equation*}
    * Update the weigth step \begin{equation*} w_i = w_i + \eta \delta x_i\end{equation*}
    
3. Update the weights \begin{equation*} w_i = w_i + \eta \nabla w_i / m\end{equation*} Here n is the learning rate and m in the number of records.(Averaging the weights to avoid large variations of training data)

In [24]:
np.random.seed(42)

Setting seed to make debugging easy

In [25]:
n_records, n_features = features.shape

In [26]:
last_loss = None

We are going to use random values for the weigths since we need them to have different starting values, breaking system. We need it to be small becuase the input to sigmoid is in the linear region near 0 and not squashed at higer and lower ends.
We will initalisze the weights from a normal distribution centered at 0. A good scale is 1/
root(n) where n is the number of input units.

In [27]:
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

In [28]:
epochs = 1000
learnrate = 0.5

In [35]:
for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output
        output = sigmoid(np.dot(x,weights))

        # TODO: Calculate the error
        error = y - output

        # TODO: Calculate the error term
        error_term = error*output*(1-output)

        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term * x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate*del_w/n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean(((out - targets) **2).astype(np.float128))
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan


In [30]:
# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.750


/Users/abhijitramesh/miniconda3/envs/gradientdescent/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until
